In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os, cv2
from tqdm import tqdm
from sklearn.model_selection import StratifiedShuffleSplit, StratifiedKFold

from tensorflow.keras import optimizers
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import *
import tensorflow_addons as tfa

In [ ]:
x_train = []
x_test = []
y_train = []

df_test = pd.read_csv('../input/plant-seedlings-classification/sample_submission.csv')


label_map = {   "Black-grass"               :0,
                "Charlock"                  :1,
                "Cleavers"                  :2,
                "Common Chickweed"          :3,
                "Common wheat"              :4,
                "Fat Hen"                   :5,
                "Loose Silky-bent"          :6,
                "Maize"                     :7,
                "Scentless Mayweed"         :8,
                "Shepherds Purse"           :9,
                "Small-flowered Cranesbill" :10,
                "Sugar beet"                :11}

dim = 300

In [ ]:
# Preparing training data
dirs = os.listdir("../input/plant-seedlings-classification/train")
for k in tqdm(range(len(dirs))):    # Directory
    files = os.listdir("../input/plant-seedlings-classification/train/{}".format(dirs[k]))
    for f in range(len(files)):     # Files
        img = cv2.imread('../input/plant-seedlings-classification/train/{}/{}'.format(dirs[k], files[f]))
        targets = np.zeros(12)
        targets[label_map[dirs[k]]] = 1 
        x_train.append(cv2.resize(img, (dim, dim)))
        y_train.append(targets)
    
y_train = np.array(y_train, np.uint8)
x_train = np.array(x_train, np.float32)

print(x_train.shape)
print(y_train.shape)

In [ ]:
# #x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, test_size=0.01, random_state=42)
# sss = StratifiedShuffleSplit(n_splits=1, test_size=0.16, random_state=42) # Want a balanced split for all the classes
# for train_index, test_index in sss.split(x_train, y_train):
#     print("Using {} for training and {} for validation".format(len(train_index), len(test_index)))
#     x_train, x_valid = x_train[train_index], x_train[test_index]
#     y_train, y_valid = y_train[train_index], y_train[test_index]

In [ ]:
from tensorflow import keras
from tensorflow.keras import backend as K

def cosine_decay_with_warmup(global_step,
                             learning_rate_base,
                             total_steps,
                             warmup_learning_rate=0.0,
                             warmup_steps=0,
                             hold_base_rate_steps=0):
    """Cosine decay schedule with warm up period.
    Cosine annealing learning rate as described in:
      Loshchilov and Hutter, SGDR: Stochastic Gradient Descent with Warm Restarts.
      ICLR 2017. https://arxiv.org/abs/1608.03983
    In this schedule, the learning rate grows linearly from warmup_learning_rate
    to learning_rate_base for warmup_steps, then transitions to a cosine decay
    schedule.
    Arguments:
        global_step {int} -- global step.
        learning_rate_base {float} -- base learning rate.
        total_steps {int} -- total number of training steps.
    Keyword Arguments:
        warmup_learning_rate {float} -- initial learning rate for warm up. (default: {0.0})
        warmup_steps {int} -- number of warmup steps. (default: {0})
        hold_base_rate_steps {int} -- Optional number of steps to hold base learning rate
                                    before decaying. (default: {0})
    Returns:
      a float representing learning rate.
    Raises:
      ValueError: if warmup_learning_rate is larger than learning_rate_base,
        or if warmup_steps is larger than total_steps.
    """

    if total_steps < warmup_steps:
        raise ValueError('total_steps must be larger or equal to '
                         'warmup_steps.')
    learning_rate = 0.5 * learning_rate_base * (1 + np.cos(
        np.pi *
        (global_step - warmup_steps - hold_base_rate_steps
         ) / float(total_steps - warmup_steps - hold_base_rate_steps)))
    if hold_base_rate_steps > 0:
        learning_rate = np.where(global_step > warmup_steps + hold_base_rate_steps,
                                 learning_rate, learning_rate_base)
    if warmup_steps > 0:
        if learning_rate_base < warmup_learning_rate:
            raise ValueError('learning_rate_base must be larger or equal to '
                             'warmup_learning_rate.')
        slope = (learning_rate_base - warmup_learning_rate) / warmup_steps
        warmup_rate = slope * global_step + warmup_learning_rate
        learning_rate = np.where(global_step < warmup_steps, warmup_rate,
                                 learning_rate)
    return np.where(global_step > total_steps, 0.0, learning_rate)


class WarmUpCosineDecayScheduler(keras.callbacks.Callback):
    """Cosine decay with warmup learning rate scheduler
    """

    def __init__(self,
                 learning_rate_base,
                 total_steps,
                 global_step_init=0,
                 warmup_learning_rate=0.0,
                 warmup_steps=0,
                 hold_base_rate_steps=0,
                 verbose=0):
        """Constructor for cosine decay with warmup learning rate scheduler.
    Arguments:
        learning_rate_base {float} -- base learning rate.
        total_steps {int} -- total number of training steps.
    Keyword Arguments:
        global_step_init {int} -- initial global step, e.g. from previous checkpoint.
        warmup_learning_rate {float} -- initial learning rate for warm up. (default: {0.0})
        warmup_steps {int} -- number of warmup steps. (default: {0})
        hold_base_rate_steps {int} -- Optional number of steps to hold base learning rate
                                    before decaying. (default: {0})
        verbose {int} -- 0: quiet, 1: update messages. (default: {0})
        """

        super(WarmUpCosineDecayScheduler, self).__init__()
        self.learning_rate_base = learning_rate_base
        self.total_steps = total_steps
        self.global_step = global_step_init
        self.warmup_learning_rate = warmup_learning_rate
        self.warmup_steps = warmup_steps
        self.hold_base_rate_steps = hold_base_rate_steps
        self.verbose = verbose
        self.learning_rates = []

    def on_batch_end(self, batch, logs=None):
        self.global_step = self.global_step + 1
        lr = K.get_value(self.model.optimizer.lr)
        self.learning_rates.append(lr)

    def on_batch_begin(self, batch, logs=None):
        lr = cosine_decay_with_warmup(global_step=self.global_step,
                                      learning_rate_base=self.learning_rate_base,
                                      total_steps=self.total_steps,
                                      warmup_learning_rate=self.warmup_learning_rate,
                                      warmup_steps=self.warmup_steps,
                                      hold_base_rate_steps=self.hold_base_rate_steps)
        K.set_value(self.model.optimizer.lr, lr)
        if self.verbose > 0:
            print('\nBatch %05d: setting learning '
                  'rate to %s.' % (self.global_step + 1, lr))

In [ ]:
datagen = ImageDataGenerator(horizontal_flip=True, 
                             vertical_flip=True)   

epochs = 100
learning_rate = 0.0001
batch_size = 16

# total_steps = x_train.shape[0] // batch_size * epochs
# warmup_steps = 0.1 * total_steps
# warm_up_lr = WarmUpCosineDecayScheduler(learning_rate_base=learning_rate,
#                                         total_steps=total_steps,
#                                         warmup_learning_rate=0.0,
#                                         warmup_steps=warmup_steps,
#                                         hold_base_rate_steps=0)

In [ ]:
# ------ TRAINING ------

skf = StratifiedKFold(n_splits=5)
for fold, (train_index, val_index) in enumerate(skf.split(x_train, np.where(y_train == 1)[1])):
    base_model = efficientnet.EfficientNetB3(input_shape=(dim, dim, 3), include_top=False, weights='imagenet') # Average pooling rswishes output dimensions
    x = base_model.output
    x = GlobalAveragePooling2D(name="avg_pool")(x)
    x = BatchNormalization()(x)
    x = Dropout(0.2, name="top_dropout")(x)
    outputs = Dense(12, activation="softmax", name="pred")(x)
    model = Model(inputs=base_model.input, outputs=outputs)
    
    callbacks = [EarlyStopping(monitor='val_loss', patience=5, verbose=0), 
                  ModelCheckpoint(f"fold_{fold}.h5", monitor='val_loss', save_best_only=True, verbose=0),
                  ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=2, verbose=0, mode='auto', epsilon=0.0001, cooldown=0, min_lr=0)]
    
    model.compile(loss='categorical_crossentropy', optimizer=optimizers.Adam(learning_rate=learning_rate), metrics=['accuracy'])
    model.fit_generator(datagen.flow(x_train[train_index], y_train[train_index], batch_size=batch_size),
                        steps_per_epoch=len(train_index)//batch_size, 
                        validation_data=datagen.flow(x_train[val_index], y_train[val_index], batch_size=batch_size), 
                        validation_steps=len(val_index)/batch_size,
                        callbacks=callbacks,
                        epochs=epochs, 
                        verbose=1)

In [ ]:
# ------ TESTING ------
for f, species in tqdm(df_test.values, miniters=100):
    img = cv2.imread('../input/plant-seedlings-classification/test/{}'.format(f))
    x_test.append(cv2.resize(img, (dim, dim)))

x_test1 = np.array(x_test, np.float32)
x_test2 = np.array([np.rot90(i, k=1) for i in x_test], np.float32)
x_test3 = np.array([np.rot90(i, k=2) for i in x_test], np.float32)
x_test4 = np.array([np.rot90(i, k=3) for i in x_test], np.float32)

# print(x_test.shape)

p_test1 = np.zeros((len(x_test), 12), dttype=float)
p_test2 = np.zeros((len(x_test), 12), dttype=float)
p_test3 = np.zeros((len(x_test), 12), dttype=float)
p_test4 = np.zeros((len(x_test), 12), dttype=float)
for fold in range(5):
    model.load_weights(f'fold_{fold}.h5')
    p_test1 += model.predict(x_test1, verbose=1)
    p_test2 += model.predict(x_test2, verbose=1)
    p_test3 += model.predict(x_test3, verbose=1)
    p_test4 += model.predict(x_test4, verbose=1)

preds = []
for i in range(len(p_test1)):
    pos = np.argmax(p_test1[i] + p_test2[i] + p_test3[i] + p_test4[i])
    preds.append(list(label_map.keys())[list(label_map.values()).index(pos)])
    
df_test['species'] = preds
df_test.to_csv('submission.csv', index=False)

In [ ]:
# df_test.head()